In [1]:
import sys
import os
path = os.path.abspath("../")
sys.path.append(path)
print(path)

d:\Uni\DSOInternship\VecRepV3


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, Sampler, random_split, Dataset, Subset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import random

from collections import defaultdict, Counter
from line_profiler import profile
from scipy.linalg import orthogonal_procrustes


import src.helpers.MetricUtilities as metrics
from functools import partial

from mtree.mtree import MTree
import mtree.mtree as mtree

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

%reload_ext autoreload

%autoreload 2

cpu


In [3]:
import sys
import os
path = os.path.abspath("../")
sys.path.append(path)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, Sampler, random_split, Dataset, Subset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import random

from collections import defaultdict, Counter
from line_profiler import profile
from scipy.linalg import orthogonal_procrustes

import src.helpers.MetricUtilities as metrics


from mtree.mtree import MTree
import mtree.mtree as mtree

import glob
from PIL import Image

import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# TODO: Put these in a separate file for MTreeUtils...
def getKNearestNeighbours(tree, point, k):
    l = tree.search(point, k)
    imgs = list(l)
    return imgs

def getMTree(data, k):
    # k: desired number of nearest neighbours
    tree = MTree(metrics.distance, max_node_size=k)
    tree.add_all(data)
    return tree

class CustomDataset(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        self.imgs_path = "../data/imdb_wiki/"
        file_list = glob.glob(self.imgs_path + "*")
        self.images = []
        for class_path in file_list:
            for dir_path in glob.glob(class_path + "/*"):
                for img_path in glob.glob(dir_path + "/*.jpg"):
                    self.images.append(img_path)

    # Defining the length of the dataset
    def __len__(self):
        return len(self.images)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        image_path = self.images[index]
        image = Image.open(image_path)
        image = transforms.functional.to_grayscale(image)

        # Applying the transform
        if self.transform:
            image = self.transform(image)
        
        return image.squeeze().to('cpu').numpy()

class CustomDatasetMStar(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        self.imgs_path = "../data/mstar/Padded_imgs/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.JPG"):
                self.data.append([img_path, class_name])
        #print(self.data)
        self.class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

    # Defining the length of the dataset
    def __len__(self):
        return len(self.data)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        data_path = self.data[index]
        image = Image.open(data_path[0])
        image = transforms.functional.to_grayscale(image)
        class_id = self.class_map[data_path[1]]
        # Applying the transform
        if self.transform:
            image = self.transform(image)
        
        return image.squeeze().to('cpu').numpy(), class_id

def get_data(size):
    transform = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])
    return CustomDataset(transform)

def get_data_MStar(size):
    transform = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])
    return CustomDatasetMStar(transform)

def mtree_ncc_query_sample_size(max_node_size=12, image_size=32, k=7, runs=2, sample_sizes=[1]):
    total_time_ncc = 0
    total_time_mtree = 0
    avg_times_ncc = []
    avg_times_mtree = []

    data = get_data_MStar(image_size)

    print(f"Average runtime of querying mtree and ncc for {k} NN over {runs} runs with image size {image_size} and max node size {max_node_size} and variable sample size")
    for i in range(len(sample_sizes)):
        print(f"NOW TRYING sample size: {sample_sizes[i]}")
        total_time_ncc = 0
        total_time_mtree = 0

        sample_indices = random.sample(range(len(data)), sample_sizes[i])
        sampled_test_data = Subset(data, sample_indices)

        testSample = [item[0] for item in sampled_test_data]
        

        tree = getMTree(testSample, max_node_size)

        # trans = transforms.Compose([transforms.Resize(img_sizes[i])])
        # t_MNIST_data = trans(MNIST_data)

        # for img in MNIST_data:
        #     img = trans(img)

        for _ in range(runs):
            index1 = np.random.randint(len(data))
            #input1=input_dataset[index1][0].squeeze().to('cpu')
            unseen_image = data[index1][0]

            start_time = time.time()
            arr = []
            for j in range(len(testSample)):
                result = ImageProducts.ncc_scaled(testSample[j], unseen_image)
                arr.append(result)
            
            unseen_img_arr = np.array(arr)
            #print(unseen_img_arr)
            imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
            end_time = time.time()

            total_time_ncc += end_time - start_time

            start_time = time.time()
            imgs = getKNearestNeighbours(tree, unseen_image, k+1)
            end_time = time.time()
            total_time_mtree += end_time - start_time

        avg_ncc = total_time_ncc / runs
        avg_mtree = total_time_mtree / runs
        avg_times_ncc.append(avg_ncc)
        avg_times_mtree.append(avg_mtree)

        # with open("test_mtree_query_sample_sizes_avg_times_ncc_.txt", "w") as file:
        #     file.write(f"avg times: {str(avg_times_ncc)}")
        
        # with open("test_mtree_query_sample_sizes_avg_times_mtree_.txt", "w") as file:
        #     file.write(f"avg times: {str(avg_times_mtree)}")

        print(f"Average runtime of ncc search: {avg_ncc:.6f} seconds")
        print(f"Average runtime of mtree search: {avg_mtree:.6f} seconds")
    # plot_data_mtree_ncc(x_axis="Sample size", title=f"Average runtime of finding {k} neighbours with image size {image_size} against sample sizes", filename="test_mtree_query_sample_sizes.png", 
    #                     varied_arr=sample_sizes, data1=avg_times_ncc, data2=avg_times_mtree, max_node_size=max_node_size)


def mtree_init_sample_size(max_node_size=12, image_size=32, k=7, runs=2, sample_sizes=[1]):
    total_time_mtree_init = 0
    avg_times_mtree_init = []

    data = get_data_MStar(image_size)

    print(f"Average runtime of initialising mtree over {runs} runs with max node size {max_node_size} and image size {image_size} and variable sample sizes")

    for i in range(len(sample_sizes)):
        print(f"NOW TRYING sample size: {sample_sizes[i]}")
        total_time_mtree_init = 0

        sample_indices = random.sample(range(len(data)), sample_sizes[i])
        sampled_test_data = Subset(data, sample_indices)

        testSample = [item[0] for item in sampled_test_data]
        #sampled_test_data[:,0]

        for _ in range(runs):
            start_time = time.time()
            tree = getMTree(testSample, max_node_size)
            end_time = time.time()
            total_time_mtree_init += end_time - start_time
            print(f"Finished one run of mtree init...")

        
        avg_mtree_init = total_time_mtree_init / runs
        avg_times_mtree_init.append(avg_mtree_init)

        
        # with open("test_mtree_init_sample_sizes_avg_times_mtree_init.txt", "w") as file:
        #     file.write(f"avg times: {str(avg_times_mtree_init)}")

        print(f"Average runtime of mtree init: {avg_mtree_init:.6f} seconds")
    
    # plot_data_mtree_init(x_axis="Sample size", title=f"Average runtime of initialising mtree with image size {image_size}, max node size {max_node_size} against sample sizes", filename="test_mtree_init_sample_sizes.png", 
    #                     varied_arr=sample_sizes, data=avg_times_mtree_init, max_node_size=max_node_size)

In [11]:
# 9466 samples
# note MStar is 368 by 368...

mtree_ncc_query_sample_size(max_node_size=15, image_size=368, k=7, runs=10, sample_sizes=[10, 100, 1000, 2000])

Average runtime of querying mtree and ncc for 7 NN over 10 runs with image size 368 and max node size 15 and variable sample size
NOW TRYING sample size: 10


KeyError: 'Padded_imgs\\ZSU_23_4'

In [12]:

k=7
data = get_data_MStar(368)
print(len(data))
sample_indices = random.sample(range(len(data)), 100)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
classes = [item[1] for item in sampled_test_data]

# class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

tree = getMTree(testSample, 12)

index1 = random.choice(sample_indices)
#input1=input_dataset[index1][0].squeeze().to('cpu')
unseen_image = data[index1][0]
unseen_image_class = data[index1][1]

arr = []
for j in range(len(testSample)):
    result = ImageProducts.ncc_scaled(testSample[j], unseen_image)
    arr.append(result)

unseen_img_arr = np.array(arr)
#print(unseen_img_arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]

num_same_class = 0

print(f"Unseen img class: {unseen_image_class}")
for i in imgProd_max_index:
    img_class = classes[i]
    print(f"Index {i}, Class {img_class}")
    if (img_class == unseen_image_class):
        num_same_class += 1

print(num_same_class)


imgs = getKNearestNeighbours(tree, unseen_image, k+1)


def imgs_to_indices(img_arr, testSample):
    ind_arr = []
    for img in img_arr:
        for i in range(len(testSample)):
            if (metrics.distance(img, testSample[i]) < 0.00001):
                ind_arr.append(i)
                break
    
    return ind_arr

ind_arr = imgs_to_indices(imgs, testSample)
print(ind_arr)
print(imgProd_max_index)


9466


KeyError: 'Padded_imgs\\SLICY'

In [4]:
import numpy as np
import cv2

def ncc(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc

    Applies NCC of the template image over the main image and returns the max value obtained.
    When the template image kernel exceeds the bounds, wraps to the other side of the main image
    """
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)

    corr = cv2.matchTemplate(mainImg, tempImg, cv2.TM_CCORR_NORMED)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corr)

    return max_val

def ncc_scaled(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc, with scaled bounds of [-1,1]
    """
    return ncc(mainImg, tempImg) * 2 - 1


class CustomDatasetMStar(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        self.imgs_path = "../data/mstar/Padded_imgs/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("\\")[-1]
            for img_path in glob.glob(class_path + "/*.JPG"):
                self.data.append([img_path, class_name])
        #print(self.data)
        self.class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

    # Defining the length of the dataset
    def __len__(self):
        return len(self.data)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        data_path = self.data[index]
        image = Image.open(data_path[0])
        image = transforms.functional.to_grayscale(image)
        class_id = self.class_map[data_path[1]]
        # Applying the transform
        if self.transform:
            image = self.transform(image)
        
        return image.squeeze().to('cpu').numpy(), class_id

k=7
data = get_data_MStar(368)
print(len(data))
sample_indices = random.sample(range(len(data)), 10)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
classes = [item[1] for item in sampled_test_data]

# class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

tree = getMTree(testSample, 12)

index1 = random.choice(sample_indices)
#input1=input_dataset[index1][0].squeeze().to('cpu')
unseen_image = data[index1][0]
unseen_image_class = data[index1][1]

arr = []
for j in range(len(testSample)):
    result = ncc_scaled(testSample[j], unseen_image)
    arr.append(result)

unseen_img_arr = np.array(arr)
#print(unseen_img_arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]

num_same_class = 0

print(f"Unseen img class: {unseen_image_class}")
for i in imgProd_max_index:
    img_class = classes[i]
    print(f"Index {i}, Class {img_class}")
    if (img_class == unseen_image_class):
        num_same_class += 1

print(num_same_class)


imgs = getKNearestNeighbours(tree, unseen_image, k+1)


def imgs_to_indices(img_arr, testSample):
    ind_arr = []
    for img in img_arr:
        for i in range(len(testSample)):
            if (metrics.distance(img, testSample[i]) < 0.00001):
                ind_arr.append(i)
                break
    
    return ind_arr

ind_arr = imgs_to_indices(imgs, testSample)
print(ind_arr)
print(imgProd_max_index)


9466
Unseen img class: 3
Index 9, Class 6
Index 8, Class 2
Index 4, Class 3
Index 7, Class 1
Index 6, Class 6
Index 3, Class 5
Index 1, Class 4
Index 2, Class 3
2


KeyboardInterrupt: 

In [10]:
import scipy.fft
import math

def ncc(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc

    Applies NCC of the template image over the main image and returns the max value obtained.
    When the template image kernel exceeds the bounds, wraps to the other side of the main image
    """
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)

    corr = cv2.matchTemplate(mainImg, tempImg, cv2.TM_CCORR_NORMED)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corr)

    return max_val

def ncc_unnormed(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc

    Applies NCC of the template image over the main image and returns the max value obtained.
    When the template image kernel exceeds the bounds, wraps to the other side of the main image
    """
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)

    corr = cv2.matchTemplate(mainImg, tempImg, cv2.TM_CCORR)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corr)

    return max_val

dataset = get_data_MStar(300)


print(len(dataset))
test_img, test_class = dataset[10]
test_img2, test_class2 = dataset[20]
m = test_img.shape[0]

ncc_score_un = ncc_unnormed(test_img, test_img2)
ncc_score = ncc(test_img, test_img2)
print(ncc_score)
print(ncc_score_un)


def ncc_fft(mainImg, tempImg):
    A = scipy.fft.fft2(test_img)
    B = scipy.fft.fft2(test_img2)

    Z = scipy.fft.ifft2(np.conj(A) * B).real

    auto_A = scipy.fft.ifft2(np.conj(A) * A).real
    auto_B = scipy.fft.ifft2(np.conj(B) * B).real
    auto_A_sqrt = np.sqrt(auto_A)
    auto_B_sqrt = np.sqrt(auto_B)
    denom = np.multiply(auto_A_sqrt, auto_B_sqrt).max() # (THIS WORKS.) (Basically uhm so the normalization amt should be the same throughout? I'm not v sure why this works...)

    return np.divide(Z, denom).max()

print(ncc_fft(test_img, test_img2))

def ncc_naive(mainImg, tempImg):
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)
    m = tempImg.shape[0]
    ncc_arr = np.ones((2*m, 2*m))
    for i in range(2 * m):
        for j in range(2 * m):
            sum = 0
            sum_norm_main = 0
            sum_norm_temp = 0
            for x in range(m):
                for y in range(m):
                    sum += mainImg[x + i][y + j] * tempImg[x][y]
                    sum_norm_main += (mainImg[x+i][y+j])**2
                    sum_norm_temp += (tempImg[x][y])**2
            
            ncc_arr[i][j] = (sum) / (math.sqrt(sum_norm_main * sum_norm_temp))
    
    return ncc_arr.max()


# print(ncc_naive(test_img, test_img2))

9466
0.8721287250518799
32470.98828125
0.8721287


## TEST NUMBA

In [11]:
import numba as nb

@nb.njit
def ncc_fft_numba(mainImg, tempImg):
    A = scipy.fft.fft2(test_img)
    B = scipy.fft.fft2(test_img2)

    Z = scipy.fft.ifft2(np.conj(A) * B).real
    # IT'S OK UP TO THIS POINT!!

    auto_A = scipy.fft.ifft2(np.conj(A) * A).real
    auto_B = scipy.fft.ifft2(np.conj(B) * B).real
    auto_A_sqrt = np.sqrt(auto_A)
    auto_B_sqrt = np.sqrt(auto_B)
    denom = np.multiply(auto_A_sqrt, auto_B_sqrt).max() # (THIS WORKS.) (Basically uhm so the normalization amt should be the same throughout? I'm not v sure why this works but not the other...)

    return np.divide(Z, denom).max()



In [ ]:
# initialising

ncc_score = ncc_fft_numba(test_img, test_img2)

In [ ]:
# Rocket-fft consistently faster! Yippee.
import time

runs = 1000
ttime = 0

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc_fft_numba(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"rocket-fft ncc fft runtime: {ttime / runs}")

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc_fft(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"ncc fft runtime: {ttime / runs}")

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"cv2 ncc runtime: {ttime / runs}")


rocket-fft ncc fft runtime: 0.03053863300019293
ncc fft runtime: 0.04279141920001712
cv2 ncc runtime: 0.09055825790012023
